In [0]:
dbutils.widgets.removeAll()

In [0]:
from pyspark.sql.functions import *
from pyspark.sql.types import *
from pyspark.sql import functions as F

In [0]:
dbutils.widgets.text("catalogo", "catalog_jptq")
dbutils.widgets.text("esquema_stage", "silver_jptq")
dbutils.widgets.text("esquema_target", "golden_jptq")
dbutils.widgets.text("tabla", "TBL_DIM_GENERO_PELICULA_T0")
dbutils.widgets.text("storageLocation", "abfss://unity-catalog-jptq@adlsjptq0126.dfs.core.windows.net")

In [0]:
catalogo = dbutils.widgets.get("catalogo")
esquema_stage = dbutils.widgets.get("esquema_stage")
esquema_target = dbutils.widgets.get("esquema_target")
tabla = dbutils.widgets.get("tabla")
storageLocation = dbutils.widgets.get("storageLocation")

In [0]:
def fn_create_table_dim_genero_pelicula(catalogo: str, esquema: str, tabla: str):
   if spark.catalog.tableExists(f"{catalogo}.{esquema_target}.{tabla}"):
      print(f"La tabla ya existe: {catalogo}.{esquema_target}.{tabla}")
   else:
      spark.sql(f"""CREATE TABLE IF NOT EXISTS {catalogo}.{esquema_target}.{tabla} (
      SK_DIM_GENERO BIGINT GENERATED ALWAYS AS IDENTITY,
      COD_GENERO_PELICULA STRING, 
      DES_GENERO_PELICULA STRING,
      FEC_CARGA DATE
      )
      USING DELTA
      LOCATION '{storageLocation}/golden-jptq/{tabla}'""")
      print(f"Tabla creada correctamente: {catalogo}.{esquema_target}.{tabla}")

In [0]:
fn_create_table_dim_genero_pelicula(catalogo,esquema_target,tabla)

In [0]:
def fn_insert_table_dim_genero_pelicula(catalogo: str, esquema: str, tabla: str):
    spark.sql(f"""MERGE INTO {catalogo}.{esquema_target}.{tabla} AS TARGET
    USING {catalogo}.{esquema_stage}.TBL_TMP_GENERO_PELICULA AS SOURCE
    ON TARGET.COD_GENERO_PELICULA = SOURCE.COD_GENERO_PELICULA
    WHEN NOT MATCHED THEN
    INSERT (
    COD_GENERO_PELICULA,
    DES_GENERO_PELICULA,
    FEC_CARGA
    )
    VALUES (
    SOURCE.COD_GENERO_PELICULA,
    SOURCE.DES_GENERO_PELICULA,
    CURRENT_DATE()
    )""")
    print(f"Insersion Correcta en {catalogo}.{esquema_target}.{tabla}")

In [0]:
fn_insert_table_dim_genero_pelicula(catalogo,esquema_target,tabla)

In [0]:
%sql
SELECT
*
FROM
catalog_jptq.golden_jptq.tbl_dim_genero_pelicula_t0